In [0]:
from pyspark.sql.functions import col, isnan, when, count, sum, min, max

In [0]:
# Nome da database e tabela
database_name = "BRONZE"
table_name = "2013_hard_drive_csv_parquet"

df = spark.table(f"{database_name}.{table_name}")

df.printSchema()


root
 |-- _c0: string (nullable = true)
 |-- date: string (nullable = true)
 |-- serial_number: string (nullable = true)
 |-- model: string (nullable = true)
 |-- capacity_bytes: string (nullable = true)
 |-- failure: string (nullable = true)
 |-- smart_1_raw: string (nullable = true)
 |-- smart_2_raw: string (nullable = true)
 |-- smart_3_raw: string (nullable = true)
 |-- smart_4_raw: string (nullable = true)
 |-- smart_5_raw: string (nullable = true)
 |-- smart_7_raw: string (nullable = true)
 |-- smart_8_raw: string (nullable = true)
 |-- smart_9_raw: string (nullable = true)
 |-- smart_10_raw: string (nullable = true)
 |-- smart_11_raw: string (nullable = true)
 |-- smart_12_raw: string (nullable = true)
 |-- smart_13_raw: string (nullable = true)
 |-- smart_15_raw: string (nullable = true)
 |-- smart_183_raw: string (nullable = true)
 |-- smart_184_raw: string (nullable = true)
 |-- smart_187_raw: string (nullable = true)
 |-- smart_188_raw: string (nullable = true)
 |-- smart_18

In [0]:
# Contar o número de valores nulos por coluna
null_counts = df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns]).collect()[0]

# Obter o número total de linhas do DataFrame
total_rows = df.count()

# Listar colunas com todos os valores nulos
columns_all_nulls = [c for c in df.columns if null_counts[c] == total_rows]


Como se possuem muitas colunas com todos os valores nulos, estas não são da valor para as análises. Assim, serão retiradas da base de dados.

In [0]:
df_cleaned = df.drop(*columns_all_nulls)
df_cleaned.limit(10).display()

_c0,date,serial_number,model,capacity_bytes,failure,smart_1_raw,smart_5_raw,smart_9_raw,smart_194_raw,smart_197_raw
3842425,2013-11-15,PL1311LAG3D8BH,Hitachi HDS5C4040ALE630,4000787030016,0,0,0,5338,25,0
3842426,2013-11-15,JK1101B9KNG2BF,Hitachi HDS722020ALA330,2000398934016,0,0,1,22473,24,0
3842427,2013-11-15,S1F0BD58,ST3000DM001,3000592982016,0,202233008,0,11738,25,0
3842428,2013-11-15,6XW01KG2,ST31500541AS,1500301910016,0,86786840,0,35125,19,0
3842429,2013-11-15,PL1311LAG2GRRH,Hitachi HDS5C4040ALE630,4000787030016,0,0,0,4679,29,0
3842430,2013-11-15,MJ1311YNG785RA,Hitachi HDS5C3030ALA630,3000592982016,0,0,0,13977,26,0
3842431,2013-11-15,S1F10BNL,ST3000DM001,3000592982016,0,194738440,0,7587,30,0
3842432,2013-11-15,JK1101B9JPHP8F,Hitachi HDS722020ALA330,2000398934016,0,0,0,23475,23,0
3842433,2013-11-15,W1F08CY4,ST3000DM001,3000592982016,0,45567528,0,14568,25,0
3842434,2013-11-15,PL1321LAG34GJH,Hitachi HDS5C4040ALE630,4000787030016,0,0,0,6028,29,0


In [0]:
# Alterar o tipo de dados das colunas
df_new = df_cleaned.withColumn("_c0", col("_c0").cast("int"))
df_new = df_new.withColumn("date", col("date").cast("date"))
df_new = df_new.withColumn("capacity_bytes", col("capacity_bytes").cast("long"))
df_new = df_new.withColumn("failure", col("failure").cast("boolean"))
df_new = df_new.withColumn("smart_1_raw", col("smart_1_raw").cast("long"))
df_new = df_new.withColumn("smart_5_raw", col("smart_5_raw").cast("integer"))
df_new = df_new.withColumn("smart_9_raw", col("smart_9_raw").cast("integer"))
df_new = df_new.withColumn("smart_194_raw", col("smart_194_raw").cast("integer"))
df_new = df_new.withColumn("smart_197_raw", col("smart_197_raw").cast("integer"))
df_new = df_new.orderBy("_c0")
df_new.limit(10).display()

_c0,date,serial_number,model,capacity_bytes,failure,smart_1_raw,smart_5_raw,smart_9_raw,smart_194_raw,smart_197_raw
0,2013-04-10,MJ0351YNG9Z0XA,Hitachi HDS5C3030ALA630,3000592982016,false,0,0,4031,26,0
1,2013-04-10,MJ0351YNG9WJSA,Hitachi HDS5C3030ALA630,3000592982016,false,0,2,4099,29,0
2,2013-04-10,MJ0351YNG9Z7LA,Hitachi HDS5C3030ALA630,3000592982016,false,0,0,3593,26,0
3,2013-04-10,MJ0351YNGAD37A,Hitachi HDS5C3030ALA630,3000592982016,false,0,0,2339,29,0
4,2013-04-10,MJ0351YNGABYAA,Hitachi HDS5C3030ALA630,3000592982016,false,0,0,2741,25,0
5,2013-04-10,MJ1311YNG7ESHA,Hitachi HDS5C3030ALA630,3000592982016,false,0,0,8723,20,0
6,2013-04-10,S2F0BE6T,ST1500DM003,1500301910016,false,23972424,0,4378,25,0
7,2013-04-10,W1F0LRXG,ST3000DM001,3000592982016,false,5538736,0,6297,25,8
8,2013-04-10,6XW099YJ,ST31500541AS,1500301910016,false,17458817,0,24737,24,0
9,2013-04-10,W1F09LPX,ST3000DM001,3000592982016,false,49517448,0,7484,29,0


Depois retiradas as colunas sem informação relevante, os tipos de dados das colunas são alterados.

In [0]:
# Criar uma expressão para contar os valores nulos em cada coluna
null_counts_3 = df_new.select([sum(when(col(c).isNull(), 1).otherwise(0)).alias(c) for c in df_new.columns])

# Mostrar os resultados
display(null_counts_3)

_c0,date,serial_number,model,capacity_bytes,failure,smart_1_raw,smart_5_raw,smart_9_raw,smart_194_raw,smart_197_raw
0,0,0,0,0,0,0,0,0,0,0


In [0]:
# Agrupar todas as colunas e contar as ocorrências
duplicates = df_new.groupBy(df_cleaned.columns).count().filter("count > 1")

# Mostrar as linhas duplicadas
print("Linhas Duplicadas:")
display(duplicates)

# Identificar todas as instâncias das linhas duplicadas
duplicate_instances = df_new.join(duplicates, on=df_new.columns, how='inner')

# Mostrar todas as instâncias das linhas duplicadas
print("Todas as Instâncias das Linhas Duplicadas:")
display(duplicate_instances)

Linhas Duplicadas:


_c0,date,serial_number,model,capacity_bytes,failure,smart_1_raw,smart_5_raw,smart_9_raw,smart_194_raw,smart_197_raw,count


Todas as Instâncias das Linhas Duplicadas:


_c0,date,serial_number,model,capacity_bytes,failure,smart_1_raw,smart_5_raw,smart_9_raw,smart_194_raw,smart_197_raw,count


Nos dois últimos trechos de código são verificados se existem valores nulos e linhas repetidas.

In [0]:

# Obter uma lista dos nomes das colunas
columns = df_new.columns

# Construir uma lista de expressões para as funções min e max
expressions = []
for c in columns:
    expressions.append(min(c).alias(f"min_{c}"))
    expressions.append(max(c).alias(f"max_{c}"))
    
# Calcular o valor mínimo e máximo de cada coluna
result = df_new.select(expressions)
result.display()

min__c0,max__c0,min_date,max_date,min_serial_number,max_serial_number,min_model,max_model,min_capacity_bytes,max_capacity_bytes,min_failure,max_failure,min_smart_1_raw,max_smart_1_raw,min_smart_5_raw,max_smart_5_raw,min_smart_9_raw,max_smart_9_raw,min_smart_194_raw,max_smart_194_raw,min_smart_197_raw,max_smart_197_raw
0,5091500,2013-04-10,2013-12-31,13H2B97AS,Z3010377,HGST HDS724040ALE640,WDC WD30EZRX,1000204886016,4000787030016,false,true,0,4410210874,0,65384,0,260425,9,50,0,65535


In [0]:
df_new_renamed = df_new.withColumnRenamed("smart_1_raw", "Read_Error_Rate") \
                        .withColumnRenamed("smart_5_raw", "Reallocated_Sectors_Count") \
                        .withColumnRenamed("smart_9_raw", "Power_On_Hours")\
                        .withColumnRenamed("smart_194_raw", "Temperature") \
                        .withColumnRenamed("smart_197_raw", "Current_Pending_Sector_Count")       

In [0]:


spark.sql("CREATE DATABASE IF NOT EXISTS PRATA")


DataFrame[]

In [0]:
spark.conf.set("spark.sql.legacy.allowCreatingManagedTableUsingNonemptyLocation","true")

permanent_table_name = "PRATA.2013_hard_drive_csv_parquet_prata"

df_new_renamed.write.mode("overwrite").format("parquet").saveAsTable(permanent_table_name)
df_new_renamed.limit(10).display()

_c0,date,serial_number,model,capacity_bytes,failure,Read_Error_Rate,Reallocated_Sectors_Count,Power_On_Hours,Temperature,Current_Pending_Sector_Count
0,2013-04-10,MJ0351YNG9Z0XA,Hitachi HDS5C3030ALA630,3000592982016,false,0,0,4031,26,0
1,2013-04-10,MJ0351YNG9WJSA,Hitachi HDS5C3030ALA630,3000592982016,false,0,2,4099,29,0
2,2013-04-10,MJ0351YNG9Z7LA,Hitachi HDS5C3030ALA630,3000592982016,false,0,0,3593,26,0
3,2013-04-10,MJ0351YNGAD37A,Hitachi HDS5C3030ALA630,3000592982016,false,0,0,2339,29,0
4,2013-04-10,MJ0351YNGABYAA,Hitachi HDS5C3030ALA630,3000592982016,false,0,0,2741,25,0
5,2013-04-10,MJ1311YNG7ESHA,Hitachi HDS5C3030ALA630,3000592982016,false,0,0,8723,20,0
6,2013-04-10,S2F0BE6T,ST1500DM003,1500301910016,false,23972424,0,4378,25,0
7,2013-04-10,W1F0LRXG,ST3000DM001,3000592982016,false,5538736,0,6297,25,8
8,2013-04-10,6XW099YJ,ST31500541AS,1500301910016,false,17458817,0,24737,24,0
9,2013-04-10,W1F09LPX,ST3000DM001,3000592982016,false,49517448,0,7484,29,0
